# **Mozart**: The ML Models Gradio Demo:

Using Gradio 3.0, this demo is designed to showcase the backend of the Artszy Platform. The Final Product would be a web app built using React, FastAPI and Python.

This demo showcases everything but the UI of the platform.

-Bharath Raj

##Installing Dependencies:

In [ ]:
nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: Please use low quality or low res <--"
  print(warning_string)
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  !git clone https://github.com/openai/CLIP
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/mfrashad/clipit.git
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom
  
  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models

import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))


## Importing Modules for Style Transfer

In [1]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import functools
import tensorflow_hub as hub


##Importing Modules for Super Resolution

In [2]:
import urllib.request
from PIL import Image
import requests
import shutil

##Importing Modules for VQGAN + Clip

In [ ]:
import torch
import clipit

##Function Definitions:

#####Style Transfer:

In [3]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def convert_img(img, quality):
  max_dim = 256 * quality
  img = tf.image.convert_image_dtype(img, tf.float32)
  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def style_transfer(content, style, quality):
    hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    content = convert_img(content, quality)
    style = convert_img(style, quality)
    stylized_image = hub_model(tf.constant(content), tf.constant(style))[0]
    return tensor_to_image(stylized_image)

#####VQGAN + Clip:

In [4]:
def generate(prompt, generation_quality, detail):
    torch.cuda.empty_cache()
    clipit.reset_settings()
    clipit.add_settings(prompts=prompt,
                        aspect='square',
                        quality=generation_quality,
                        use_pixeldraw=False,
                        use_clipdraw=False,
                        make_video=False,
                        iterations={'draft':30, 'normal': 80, 'better': 150}[detail])
    settings = clipit.apply_settings()
    clipit.do_init(settings)
    clipit.do_run(settings)

    return 'output.png'

#####Super Resolution:

In [5]:
def enhance(path):
  Image.fromarray(path.astype('uint8'), 'RGB').save('temp.jpg', 'jpeg')
  r = requests.post(
    "https://api.deepai.org/api/waifu2x",
    files={
        'image': open('temp.jpg', 'rb')
    },
    headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'}
  )
  urllib.request.urlretrieve(r.json()["output_url"],"highres.png")
  img = Image.open("highres.png")
  return img
  

#Gradio Interface:



In [6]:
%%capture
!pip install gradio

In [7]:
import gradio as gr

##Style Transfer:

In [29]:
%%capture

st_description = """
<center>
<img src=https://i.imgur.com/goqyoWH.png width=700px>
</center>
"""

content = gr.inputs.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="content")
style = gr.inputs.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="Style")
quality = gr.inputs.Radio(choices=[1, 2, 3, 4], label="Quality")

editor = gr.Interface(style_transfer, 
                      inputs=[content, style, quality], 
                      description = st_description,
                      article = "Style transfer is a computer vision technique"
                      +"that allows us to recompose the content of an image in the style of another",
                      outputs=["image"],
                      allow_flagging = "never",
                      live=False)

## VQGAN + CLIP:

In [34]:
%%capture

ga_description = """
<center>
<img src=https://i.imgur.com/lolHhBs.png width=700px>
</center>
"""
prompt = gr.inputs.Textbox(default="Flying Cars", label="Text Prompt")
generation_quality = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Resolution")
detail = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Detail")


generator = gr.Interface(generate, 
                         inputs=[prompt, generation_quality, detail], 
                         outputs=['image'], 
                         description = ga_description,
                         enable_queue=True, 
                         live=False)

## Super Resolution:

In [33]:
%%capture

sr_description = """
<center>
<img src=https://i.imgur.com/RhjYIZh.png width=700px>
</center>
"""
original = gr.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="content")
enhancer = gr.Interface(enhance, 
                      inputs=[original], 
                      description = sr_description,
                      article = "2x your image resolution",
                      outputs=["image"],
                      allow_flagging = "never",
                      live=False)

## Combined Interface

In [ ]:
combined = gr.TabbedInterface([generator, editor, enhancer], ["Generate Art", "Style Transfer", "Super Resolution"])
combined.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://39311.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Clean Cache:

In [ ]:
# torch.cuda.empty_cache()